# Загрузка библиотек

In [ ]:
!pip install python-telegram-bot --upgrade                                      # установка библиотеки для подключения чатбота к телеграму

     |████████████████████████████████| 490 kB 5.3 MB/s 
     |████████████████████████████████| 58 kB 4.9 MB/s 
     |████████████████████████████████| 428 kB 38.4 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.


In [ ]:
import json                                                                     # модуль чтения файлов
from sklearn.linear_model import LogisticRegression                             # импортируем классификатор
from sklearn.feature_extraction.text import CountVectorizer                     # импортируем векторизатор текстов
from sklearn.model_selection import train_test_split                            # импортируем модуль разделения выборки на проверочную и обучающую

import logging                                                                  #

from telegram import Update, ForceReply                                         # 
from telegram.ext import Updater, CommandHandler, MessageHandler                #
from telegram.ext import Filters, CallbackContext                               #

In [ ]:
from google.colab import drive                                                  # подключаем гугл диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/chat_bot/BOT_CONFIG.json', 'r') as f:        # загружаем файл с конфигурацией бота
  BOT_CONFIG = json.load(f)

# Подготовка базы сообщений

In [ ]:
X = []                                                                          # создадим переменную под список вопросов
y = []                                                                          # список потребностей

for intent in BOT_CONFIG['intents'].keys():
    try:
      if intent != 'other':
        for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(example)
            y.append(intent)
    except:
      print(BOT_CONFIG['intents'][intent])

In [ ]:
len(X), len(y)                                                                  # размер базы ответ-вопрос

(1185, 1185)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)            # разделим выборку на обучающую и проверочную

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2, 4), analyzer='char')               # соберем уникальные ключи и создадим словарь частотности
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
len(vectorizer.get_feature_names())

6798

In [ ]:
clf = LogisticRegression().fit(X_train_vectorized, y_train)                     # обучаем 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
clf.score(X_train_vectorized, y_train)                                          # выводим точность на тренировочных данных

0.859704641350211

In [ ]:
clf.score(X_test_vectorized, y_test)                                            # выводим точность на проверочных данных

0.1729957805907173

In [ ]:
clf.predict(vectorizer.transform(['откуда ты?']))

array(['whereareyoufrom'], dtype='<U22')

In [ ]:
def get_intent_by_model(text):                                                  # функция возвращает потребность пользователя
  return clf.predict(vectorizer.transform([text]))[0]

# Вызов бота

In [ ]:
def bot(input_text):                                                            # функция принимает на вход вопрос и выдает ответ из обученного классификатора
  intent = get_intent_by_model(input_text)
  return random.choice(BOT_CONFIG['intents'][intent]['responces'])

In [ ]:
input_text = ''                                                                 # сюда сохраним пользовательский ввод данных
while input_text != 'Завершить работу':                                         # запустим локально чат бота
  input_text = input()
  print(bot(input_text))

привет!
доброго дня
как дела?
замечательно
что предложите посмотреть?
круиз по джунглям
как погода?
ожидаются дождь и ветер
Завершить работу
чтобы завершить диалог. необходимо в точности набрать текст "завершить работу"


# Подключение бота к телеграмм

In [ ]:
# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1976207298:AAEIFFVucNIrus93SxUDhPOhh-y0fX1Hgc4")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()                                                                          # запускаем бота в телеграмме

2021-09-27 01:52:34,842 - apscheduler.scheduler - INFO - Scheduler started
2021-09-27 01:53:32,726 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-09-27 01:53:32,730 - apscheduler.scheduler - INFO - Scheduler has been shut down
